In [3]:
import csv
import os
import requests
import json
import time
import io
import datetime;
import pandas as pd
from tqdm import tqdm
import magic
from pandas import ExcelWriter

os.chdir(r"D:\Users\snangare\Sandesh\API")
flag_list = []


def authenticate():
    
    endpoint = 'https://amgensbx-etmf.veevavault.com/api/v20.3/auth' 
    parameters = {
        'username':'snangare@amgensbx.com',
        'password':'Test@54321'
    }
    
    try: 
        response = requests.post(endpoint, data=parameters).json()
        if response['responseStatus'] != 'SUCCESS':
            raise Exception()
        else:
            return response['sessionId']
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)
        
def fetch_documents(session_id,endpoint,doc_name,major_version,minor_version,doc_id):
    
    headers = {
        'Authorization': session_id
    }
    
    response = requests.get(endpoint, headers=headers)
    
    file= doc_name+'_'+ doc_id +'_'+ major_version+'_'+ minor_version+'_'+'audit_trail_rendition__c'
    
    try :
        if response.json():
            
            response = response.json()
            
            if response['responseStatus'] != 'SUCCESS' and "Cannot find rendition [audit_trail_rendition__c]" in response['errors'][0]['message']:
                flag_list.append('No')
            elif response['responseStatus'] != 'SUCCESS' and "Document not found" in response['errors'][0]['message']:
                flag_list.append('Document not found')
            else:
                flag_list.append('Other') 

    except:
        flag_list.append('Yes')
        
def execute():
    session_id = authenticate()
    
    df = pd.read_csv('CK Transfer Wave-1 All Version.csv')
    os.chdir("exports/audit_trial_renditions")
    
    try:
        
        for ind in tqdm(df.index): 
                
            doc_id = str(df['Document ID'][ind])
            doc_name = str(df['Document Number'][ind])
            major_version = str(df['Major Version Number'][ind])
            minor_version = str(df['Minor Version Number'][ind])

            URI = 'https://amgensbx-etmf.veevavault.com/api/v20.3/'
            action = 'objects/documents/'+str(doc_id)+'/versions/'+major_version+'/'+minor_version+'/renditions/audit_trail_rendition__c'
            endpoint = URI + action

            fetch_documents(session_id,endpoint,doc_name,major_version,minor_version,doc_id)

        df['isAvailable?'] = flag_list
        writer = ExcelWriter(r'D:\Users\snangare\Sandesh\API\logs\attachment_check\check_log.xlsx')
        df.to_excel(writer,'check_log',index=False)
        writer.save()
        
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)  
        
execute()   

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [1:10:32<00:00,  4.73it/s]
D:\Users\snangare\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:937: UserWarning: Ignoring URL 'https://amgen-etmf.veevavault.com/ui/' with link or location/anchor > 2079 characters since it exceeds Excel's limit for URLS
  (force_unicode(url), max_url))
D:\Users\snangare\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://amgen-etmf.veevavault.com/ui/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
